# Python - intermediate

- profiling
- memory management
- threading
- async.io
- socket
- regular expression

## Profiling

In [1]:
import cProfile
import functools

def fibo(n):
    if n == 1:
        return 1
    if n == 2:
        return 1
    return fibo(n - 2) + fibo(n - 1)

# with cache
cache = {}

def fibo_with_cache(n):
    if n in cache:
        return cache[n]
    if n == 1:
        v = 1
    elif n == 2:
        v = 2
    else:
        v = fibo_with_cache(n - 2) + fibo_with_cache(n - 1)
    cache[n] = v
    return v

@functools.lru_cache(maxsize=50)
def fibo2(n):
    if n == 1:
        return 1
    if n == 2:
        return 1
    return fibo2(n - 2) + fibo2(n - 1)

cProfile.run('fibo(30)')
cProfile.run('fibo_with_cache(30)')
cProfile.run('fibo2(30)')

         1664082 function calls (4 primitive calls) in 6.977 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
1664079/1    6.977    0.000    6.977    6.977 <ipython-input-1-35f1d2d7da4e>:4(fibo)
        1    0.000    0.000    6.977    6.977 <string>:1(<module>)
        1    0.000    0.000    6.977    6.977 {built-in method builtins.exec}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}


         60 function calls (4 primitive calls) in 0.000 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
     57/1    0.000    0.000    0.000    0.000 <ipython-input-1-35f1d2d7da4e>:14(fibo_with_cache)
        1    0.000    0.000    0.000    0.000 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 {built-in method builtins.exec}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' object

## Memory management

In [2]:
print(id(-6))
print(id(-5))
print(id(1))
print(id(2-1))
print(id(256))
print(id(257))

140494154057456
10914304
10914496
10914496
10922656
140494154058320


In [3]:
print(id(-6))
print(id(-5))
print(id(1))
print(id(2-1))
print(id(256))
print(id(257))

140494154057456
10914304
10914496
10914496
10922656
140494154057872


In [4]:
import sys
print(sys.getrefcount(1))
a=1
print(sys.getrefcount(1))

3383
3384


In [5]:
class A:
    pass
print(id(A()))
print(id(A()))

140494153699280
140494153699112


In [6]:
a = A()
print(sys.getrefcount(a))
b = a
print(sys.getrefcount(a))

2
3


In [7]:
import gc
print("gc.isenabled()", gc.isenabled())
print("gc.get_threshold()", gc.get_threshold())
print("gc.get_count()", gc.get_count())

gc.isenabled() True
gc.get_threshold() (700, 10, 10)
gc.get_count() (265, 4, 3)


In [8]:
gc.collect()

0

In [9]:
print("gc.get_count()", gc.get_count())

gc.get_count() (343, 0, 0)


## Socket programming

In [10]:
import socket
import ssl

In [11]:
HOST = 'example.com'  # The server's hostname or IP address
PORT = 80                # The port used by the server

In [12]:
IP = socket.gethostbyname(HOST)
print(IP)

93.184.216.34


In [13]:
MESSAGE = \
b"""
GET / HTTP/1.1
User-Agent: Mozilla/4.0 (compatible; MSIE5.01; Windows NT)
Host: example.com
Accept-Language: en-us
Accept-Encoding: identity
Connection: keep-alive
Cache-Control: private, max-age=0, no-cache
Pragma: no-cache
Expires: Wed, 21 Oct 2015 07:28:00 GMT

"""

In [14]:
with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    s.connect((HOST, PORT))
    s.sendall(MESSAGE)
    data = s.recv(1024*1024)
    print(data)

print(data.decode('UTF-8'))

b'HTTP/1.1 200 OK\r\nCache-Control: max-age=604800\r\nContent-Type: text/html; charset=UTF-8\r\nDate: Fri, 25 Oct 2019 16:32:01 GMT\r\nEtag: "3147526947+ident"\r\nExpires: Fri, 01 Nov 2019 16:32:01 GMT\r\nLast-Modified: Thu, 17 Oct 2019 07:18:26 GMT\r\nServer: ECS (sjc/4E68)\r\nVary: Accept-Encoding\r\nX-Cache: HIT\r\nContent-Length: 1256\r\n\r\n<!doctype html>\n<html>\n<head>\n    <title>Example Domain</title>\n\n    <meta charset="utf-8" />\n    <meta http-equiv="Content-type" content="text/html; charset=utf-8" />\n    <meta name="viewport" content="width=device-width, initial-scale=1" />\n    <style type="text/css">\n    body {\n        background-color: #f0f0f2;\n        margin: 0;\n        padding: 0;\n        font-family: -apple-system, system-ui, BlinkMacSystemFont, "Segoe UI", "Open Sans", "Helvetica Neue", Helvetica, Arial, sans-serif;\n        \n    }\n    div {\n        width: 600px;\n        margin: 5em auto;\n        padding: 2em;\n        background-color: #fdfdff;\n     

In [15]:
import socket
import ssl

MESSAGE = \
b"""
GET / HTTP/1.1
User-Agent: Mozilla/4.0 (compatible; MSIE5.01; Windows NT)
Host: google.com
Accept-Language: en-us
Accept-Encoding: identity
Connection: keep-alive
Cache-Control: private, max-age=0, no-cache
Pragma: no-cache
Expires: Wed, 21 Oct 2015 07:28:00 GMT

"""
HOST = "google.com"
PORT = 443         # The port used by the server

context = ssl.create_default_context()

with socket.create_connection((HOST, PORT)) as sock:
    with context.wrap_socket(sock, server_hostname=HOST) as ssock:
        ssock.sendall(MESSAGE)
        data = ssock.recv(1024)

print(data.decode('UTF-8'))

HTTP/1.1 301 Moved Permanently
Location: https://www.google.com/
Content-Type: text/html; charset=UTF-8
Date: Fri, 25 Oct 2019 16:32:02 GMT
Expires: Sun, 24 Nov 2019 16:32:02 GMT
Cache-Control: public, max-age=2592000
Server: gws
Content-Length: 220
X-XSS-Protection: 0
X-Frame-Options: SAMEORIGIN
Alt-Svc: quic=":443"; ma=2592000; v="46,43",h3-Q048=":443"; ma=2592000,h3-Q046=":443"; ma=2592000,h3-Q043=":443"; ma=2592000

<HTML><HEAD><meta http-equiv="content-type" content="text/html;charset=utf-8">
<TITLE>301 Moved</TITLE></HEAD><BODY>
<H1>301 Moved</H1>
The document has moved
<A HREF="https://www.google.com/">here</A>.
</BODY></HTML>



## Threading

In [16]:
import concurrent.futures
import time

N_THREADS = 5
fake_data = [x for x in range(N_THREADS)]
counter = 0

def increment_counter(fake_value):
    global counter
    for _ in range(1000000):
        counter += 1

with concurrent.futures.ThreadPoolExecutor(max_workers=N_THREADS) as executor:
    executor.map(increment_counter, fake_data)

print("counter: {}".format(counter))

counter: 2539832


In [17]:
import dis

dis.dis(increment_counter)

 10           0 SETUP_LOOP              24 (to 26)
              2 LOAD_GLOBAL              0 (range)
              4 LOAD_CONST               1 (1000000)
              6 CALL_FUNCTION            1
              8 GET_ITER
        >>   10 FOR_ITER                12 (to 24)
             12 STORE_FAST               1 (_)

 11          14 LOAD_GLOBAL              1 (counter)
             16 LOAD_CONST               2 (1)
             18 INPLACE_ADD
             20 STORE_GLOBAL             1 (counter)
             22 JUMP_ABSOLUTE           10
        >>   24 POP_BLOCK
        >>   26 LOAD_CONST               0 (None)
             28 RETURN_VALUE


In [18]:

import concurrent.futures
import threading
import time

lock = threading.Lock()

N_THREADS = 5
fake_data = [x for x in range(N_THREADS)]
counter = 0

def increment_counter(fake_value):
    global counter
    for _ in range(1000000):
        lock.acquire()
        counter += 1
        lock.release()

with concurrent.futures.ThreadPoolExecutor(max_workers=N_THREADS) as executor:
    executor.map(increment_counter, fake_data)

print("counter: {}".format(counter))

counter: 5000000


In [19]:
import threading
import time
import logging

logging.basicConfig(level=logging.DEBUG,
                    format='(%(threadName)-9s) %(message)s',)

def consumer(cv):
    logging.debug('Consumer thread started ...')
    with cv:
        logging.debug('Consumer waiting ...')
        cv.wait()
        logging.debug('Consumer consumed the resource')

def producer(cv):
    logging.debug('Producer thread started ...')
    with cv:
        logging.debug('Making resource available')
        logging.debug('Notifying to all consumers')
        cv.notifyAll()

if __name__ == '__main__':
    condition = threading.Condition()
    cs1 = threading.Thread(name='consumer1', target=consumer, args=(condition,))
    cs2 = threading.Thread(name='consumer2', target=consumer, args=(condition,))
    pd = threading.Thread(name='producer', target=producer, args=(condition,))

    cs1.start()
    time.sleep(2)
    cs2.start()
    time.sleep(2)
    pd.start()

(consumer1) Consumer thread started ...
(consumer1) Consumer waiting ...
(consumer2) Consumer thread started ...
(consumer2) Consumer waiting ...
(producer ) Producer thread started ...
(producer ) Making resource available
(producer ) Notifying to all consumers
(consumer2) Consumer consumed the resource
(consumer1) Consumer consumed the resource


## asyncio

In [20]:
# apply nest_asyncio
# this is only for jupyter notebook environment
import nest_asyncio
nest_asyncio.apply()

In [21]:
import asyncio

counter = 0

async def increment_counter(id):
    global counter
    print(f"before sleep id={id}")
    await asyncio.sleep(1)
    counter += 1
    print(f"after sleep id={id}")

async def run_all():
    await asyncio.gather(*[increment_counter(i) for i in range(10)])

print(f"before run counter={counter}")

loop = asyncio.get_event_loop()
loop.run_until_complete(run_all())

print(f"after run counter={counter}")


before run counter=0
before sleep id=7
before sleep id=9
before sleep id=8
before sleep id=5
before sleep id=1
before sleep id=0
before sleep id=6
before sleep id=4
before sleep id=2
before sleep id=3
after sleep id=7
after sleep id=9
after sleep id=8
after sleep id=5
after sleep id=1
after sleep id=0
after sleep id=6
after sleep id=4
after sleep id=2
after sleep id=3
after run counter=10


In [29]:
import asyncio

counter = 0

async def increment_counter(id):
    global counter
    print(f"before sleep id={id}")
    await asyncio.sleep(1)
    counter += 1
    raise Exception(id)
    print(f"after sleep id={id}")

async def run_all():
    try:
        results = await asyncio.gather(*[increment_counter(i) for i in range(10)])
    except Exception as e:
        print("Got exception", e)
        pass

print(f"before run counter={counter}")

loop = asyncio.get_event_loop()
loop.run_until_complete(run_all())

print(f"after run counter={counter}")


before run counter=0
before sleep id=8
before sleep id=4
before sleep id=0
before sleep id=9
before sleep id=3
before sleep id=2
before sleep id=6
before sleep id=1
before sleep id=7
before sleep id=5
Got exception 8
after run counter=10


## Regular expressions

In [23]:
import re
text = "abc123,aaa bb cc"

In [24]:
r1 = re.findall(r"\w+", text)
print(r1)

['abc123', 'aaa', 'bb', 'cc']


In [25]:
r2 = re.findall(r"^\w+", text)
print(r2)

['abc123']


In [26]:
email_pattern = r"(^[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+$)"
emails = [
    "user@example.com",
    "user@example@com",
]
for email in emails:
    result = re.match(email_pattern, email)
    if result:
        print(f"{email} is an email address.")
    else :
        print(f"{email} is NOT an email address.")

user@example.com is an email address.
user@example@com is NOT an email address.


In [27]:
korean_cell_phone_pattern = r"010-\d{4}-\d{4}"
phone_numbers = [
    "010-1234-5678",
    "010-123-456"
]
for num in phone_numbers:
    result = re.match(korean_cell_phone_pattern, num)
    if result:
        print(f"{num} is an phone number.")
    else :
        print(f"{num} is NOT an phone number.")

010-1234-5678 is an phone number.
010-123-456 is NOT an phone number.


In [28]:
import re

text = "My name is Paul. Nice to meet you."
text2 = re.sub(r"My name is (.+). Nice to meet you.", r"Hello, \1. Good to see you, \1.", text)
print(text2)

Hello, Paul. Good to see you, Paul.


## References

- https://docs.python.org/3/library/profile.html
- https://realpython.com/python-memory-management/
- https://stackify.com/python-garbage-collection/
- https://realpython.com/python-sockets/
- https://realpython.com/intro-to-python-threading/
- https://realpython.com/python-concurrency/